In [1]:
#Importation

import tkinter as tk
import random
from PIL import Image, ImageTk

In [2]:
#Initailisation 

MIN_RESISTANCE = 10
MAX_RESISTANCE= 1000
MIN_CAPACITANCE = 1
MAX_CAPACITANCE = 100
MIN_INDUCTANCE = 0.1
MAX_INDUCTANCE = 10


In [3]:
#Initialisation de la population 

def initialize_population(population_size):
    # Liste qui contient tous les individus 
    population = []
    for _ in range(population_size):
        # Dictionnaire pour representer un individu 
        individual = {
            'resistance': random.uniform(MIN_RESISTANCE, MAX_RESISTANCE),
            'capacitance': random.uniform(MIN_CAPACITANCE, MAX_CAPACITANCE),
            'inductance': random.uniform(MIN_INDUCTANCE, MAX_INDUCTANCE)
        }
        population.append(individual)
    return population

In [4]:
#Fonction Fitness 
def evaluate_circuit(individual):
    resistance_fitness = 1000 / individual['resistance']  # la fitness de la resistance basée sur la résistance inverse
    current = 10 / individual['resistance']
    power_consumption = resistance_fitness * current ** 2 #LOI Joul

    capacitance_fitness = individual['capacitance'] * 5 # la fitness de la  capacitance
    inductance_fitness = 2 * individual['inductance']  # la fitness de l'inductance

    # Inclure current et power_consumption dans le calcul de la fitness totale
    total_fitness = (
        resistance_fitness * 0.2 +  # Poids pour la résistance
        capacitance_fitness * 0.2 +  # Poids pour la capacitance
        inductance_fitness * 0.2 +  # Poids pour l'inductance
        current * 0.2 +  # Poids pour le courant
        power_consumption * 0.2  # Poids pour la consommation d'énergie
    )
    return total_fitness


In [5]:
# Fonction de croisement entre deux parents avec une probabilité donnée

def crossover(parent1, parent2):
    # Vérifier si le croisement a lieu en fonction de la probabilité
    crossover_probability = 0.7
    if random.random() <= crossover_probability:
        # Créer un enfant en prenant la moyenne des valeurs de chaque propriété
        child = {
            'resistance': (parent1['resistance'] + parent2['resistance']) / 2,
            'capacitance': (parent1['capacitance'] + parent2['capacitance']) / 2,
            'inductance': (parent1['inductance'] + parent2['inductance']) / 2
        }
        
        # Retourner l'enfant résultant du croisement
        return child
    else:
        # Si le croisement n'a pas lieu, retourner l'un des parents (au hasard)
        return random.choice([parent1, parent2])

In [6]:
# Fonction de mutation pour un individu


def mutate(individual):
    # Vérifier si la mutation a lieu pour la résistance
    mutation_probability= 0.2
    if random.random() < mutation_probability:
        # Muter la résistance en une nouvelle valeur aléatoire dans l'intervalle spécifié
        individual['resistance'] = random.uniform(MIN_RESISTANCE, MAX_RESISTANCE)
    
    # Vérifier si la mutation a lieu pour la capacitance
    if random.random() < mutation_probability:
        # Muter la capacitance en une nouvelle valeur aléatoire dans l'intervalle spécifié
        individual['capacitance'] = random.uniform(MIN_CAPACITANCE, MAX_CAPACITANCE)
    
    # Vérifier si la mutation a lieu pour l'inductance
    if random.random() < mutation_probability:
        # Muter l'inductance en une nouvelle valeur aléatoire dans l'intervalle spécifié
        individual['inductance'] = random.uniform(MIN_INDUCTANCE, MAX_INDUCTANCE)
    
    # Retourner l'individu résultant de la mutation
    return individual

In [7]:
# Fonction de sélection par la méthode de le roulette 

def roulette_wheel_selection(population, total_fitness):
    # Générer un point de sélection aléatoire sur la roue
    pick = random.uniform(0, total_fitness)
    
    # Initialiser une variable pour suivre la position actuelle sur la roue
    current = 0
    
    # Parcourir la population
    for individual in population:
        # Ajouter la valeur de fitness de l'individu actuel à la position actuelle sur la roue
        current += evaluate_circuit(individual)
        
        # Vérifier si la position actuelle dépasse le point de sélection
        if current > pick:
            # Si c'est le cas, retourner l'individu actuel
            return individual
    
    # Si aucun individu n'est sélectionné avant la fin de la boucle, retourner le dernier individu de la population
    return population[-1]


In [8]:
# Fonction pour exécuter un algorithme génétique

def genetic_algorithm(population_size, generations):
    # Liste pour stocker les détails de chaque génération
    details = []  
    
    # Initialiser la population initiale
    population = initialize_population(population_size)
    
    # Boucle à travers les générations
    for gen in range(generations):
        # Calculer la somme totale de fitness de la population actuelle
        total_fitness = sum(evaluate_circuit(individual) for individual in population)
        
        # Sélectionner des individus pour la prochaine génération en utilisant la méthode de la roue de la fortune
        selected = [roulette_wheel_selection(population, total_fitness) for _ in range(population_size)]
        
        # Initialiser une liste pour stocker les enfants résultant du croisement et de la mutation
        children = []
        
        # Boucle à travers les individus sélectionnés par paire pour le croisement
        for i in range(0, population_size, 2):
            # Effectuer le croisement entre les paires d'individus sélectionnés
            child1 = crossover(selected[i], selected[i + 1])
            child2 = crossover(selected[i + 1], selected[i])
            
            # Appliquer la mutation aux enfants
            child1 = mutate(child1)
            child2 = mutate(child2)
            
            # Ajouter les enfants à la liste
            children.extend([child1, child2])
        
        # Remplacer la population actuelle par les enfants
        population = children[:]
        
        # Enregistrer les détails de chaque génération
        best_individual = max(population, key=evaluate_circuit)
        avg_fitness = sum(evaluate_circuit(individual) for individual in population) / population_size
        details.append({
            'generation': gen,
            'best_individual': best_individual,
            'average_fitness': avg_fitness
        })
    
    # Retourner le meilleur individu de la dernière génération et les détails de chaque génération
    return best_individual, details


In [9]:
# Initialisation des variables pour stocker des images
resistance_tk_img = None
capacitance_tk_img = None
inductance_tk_img = None

# Fonction principale pour exécuter l'algorithme génétique
def run_genetic_algorithm():
    global resistance_tk_img, capacitance_tk_img, inductance_tk_img
    
    # Récupération des paramètres de l'utilisateur depuis l'interface graphique
    population_size = int(population_entry.get())
    generations = int(generations_entry.get())
    
    # Exécution de l'algorithme génétique pour obtenir le meilleur circuit et les détails de chaque génération
    best_circuit, gen_details = genetic_algorithm(population_size, generations)
    
    # Affichage du résultat dans l'interface graphique
    result_label.config(text=f"Meilleur circuit - Résistance: {best_circuit['resistance']:.2f} Ω, "
                             f"Capacité: {best_circuit['capacitance']:.2f} F, "
                             f"Inductance: {best_circuit['inductance']:.2f} H, "
                             f"Fitness : {evaluate_circuit(best_circuit):.2f}", font=("Helvetica", 10, "bold"), foreground="red")

    # Création d'une fenêtre séparée pour afficher les détails sous forme de tableau
    details_window = tk.Toplevel(root)
    details_window.title("Détails par génération")

    # Création du tableau avec en-têtes et images
    table = tk.Frame(details_window)
    table.pack(padx=10, pady=10)
    
    # Chargement des images pour les en-têtes du tableau
    resistance_image_path = "./image/resistance.png"
    capacitance_image_path = "./image/condensateur.png"
    inductance_image_path = "./image/bobine.png"

    resistance_img = Image.open(resistance_image_path)
    capacitance_img = Image.open(capacitance_image_path)
    inductance_img = Image.open(inductance_image_path)

    # Redimensionnement des images pour les en-têtes
    resistance_tk_img = ImageTk.PhotoImage(resistance_img.resize((20, 20)))
    capacitance_tk_img = ImageTk.PhotoImage(capacitance_img.resize((20, 20)))
    inductance_tk_img = ImageTk.PhotoImage(inductance_img.resize((20, 20)))

    # Définition des en-têtes du tableau
    headers = ["Génération", "Resistance", "Condensateur", "Inductance", "Fitness"]
    for col, header in enumerate(headers):
        if col == 1:
            label = tk.Label(table, text="Resistance", image=resistance_tk_img, relief=tk.RIDGE, width=70, compound="top")
        elif col == 2:
            label = tk.Label(table, text="Condensateur", image=capacitance_tk_img, relief=tk.RIDGE, width=70, compound="top")
        elif col == 3:
            label = tk.Label(table, text="Inductance", image=inductance_tk_img, relief=tk.RIDGE, width=70, compound="top")
        else:
            label = tk.Label(table, text=header, relief=tk.RIDGE, width=20)
        label.grid(row=0, column=col)

    # Remplissage du tableau avec les détails de chaque génération
    for gen, detail in enumerate(gen_details, start=1):
        tk.Label(table, text=gen).grid(row=gen, column=0)
        tk.Label(table, text=f"{detail['best_individual']['resistance']:.2f}").grid(row=gen, column=1)
        tk.Label(table, text=f"{detail['best_individual']['capacitance']:.2f}").grid(row=gen, column=2)
        tk.Label(table, text=f"{detail['best_individual']['inductance']:.2f}").grid(row=gen, column=3)
        tk.Label(table, text=f"{detail['average_fitness']:.2f}").grid(row=gen, column=4)
       

# Initialisation de la fenêtre principale et des autres éléments graphiques
        
root = tk.Tk()  # Crée une instance de la classe Tk
root.title("Algorithme Génétique pour Conception de Circuits Électroniques")  
root.minsize(400, 400)  

# But de l'algorithme génétique
goal_label = tk.Label(root, text="But de l'algorithme génétique:", font=("Helvetica", 12, "bold"))
goal_label.pack()  

goal_text = """
Cet algorithme optimise les composants d'un circuit pour améliorer l'efficacité 
énergétique d'un panneau solaire, assurant une production stable et réduisant 
les pertes d'énergie pour une durée de vie prolongée du système.
"""
goal_description = tk.Label(root, text=goal_text, wraplength=300)  # Crée un widget Label avec du texte et une longueur de ligne maximale
goal_description.pack() 

# Image de circuit
circuit_img = Image.open("./image/circuit-electrique.png")  # Charge une image de circuit depuis un fichier
circuit_tk_img = ImageTk.PhotoImage(circuit_img.resize((60, 60)))  

circuit_label = tk.Label(root, image=circuit_tk_img)  
circuit_label.pack()  
    
# Éléments pour la taille de la population
population_label = tk.Label(root, text="Taille de la population :")  # Crée un widget Label pour afficher du texte
population_label.pack() 
population_entry = tk.Entry(root)  
population_entry.pack()  

# Éléments pour le nombre de générations
generations_label = tk.Label(root, text="Nombre de générations :")  # Crée un widget Label pour afficher du texte
generations_label.pack()  


generations_entry = tk.Entry(root)  
generations_entry.pack()  

# Bouton pour lancer l'algorithme génétique
run_button = tk.Button(root, text="Lancer l'algorithme génétique", command=run_genetic_algorithm)
run_button.pack()  

# Étiquette pour afficher le résultat de l'algorithme génétique
result_label = tk.Label(root, text="")
result_label.pack() 

# Étiquette pour afficher d'autres détails
details_label = tk.Label(root, text="")
details_label.pack() 

# Démarrage de la boucle principale de l'interface graphique
root.mainloop()
